In [42]:
import twitter
import requests
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class Twitter(object):
    def __init__(self):
        consumer_key = 'LFTXnbNVhCtLV2T2dG0yA'
        consumer_secret = '2DX6L81tUHP8Clxz8FvuXcaMvHTAXuUkXbzz7y6U'
        access_token = '77452634-avWq03PwpORlOtKoa4ctg082fTdFlBbKT0ihC8XZw'
        access_token_secret = 'cnopinzlsPRi6zBOMGdAVf0EuMM97eCQF5UPoyTyY'

        self.api = twitter.Api(consumer_key=consumer_key,
                               consumer_secret=consumer_secret,
                               access_token_key=access_token,
                               access_token_secret=access_token_secret)
        
    def get_tweet(self, id):
        return self.api.GetStatus(id).AsDict()

In [3]:
def get_geoname_data(geoname):
    return requests.get(u'http://api.geonames.org/postalCodeSearchJSON?placename={0}&maxRows=100&username=demo&style=FULL'.format(geoname)).json()

In [60]:
polygone = {
    'point': [(2.5454, 51.0891, 5), (8.2306, 48.96722, 5), (2.5328, 42.3327, 5), (-4.79556, 48.41278, 5)]
}

In [65]:
def make_graph(polygones):
    fig = plt.figure()
    ax = Axes3D(fig)
    
    minx= 0
    maxx= 0
    miny = 0
    maxy = 0
    minz = 0
    maxz = 0
    
    for poly in polygones:
        x = [p[0] for p in poly['point']]
        y = [p[1] for p in poly['point']]
        z = [p[2] for p in poly['point']]
        
        minx= min([minx, min(x)])
        maxx= max([maxx, max(x)])
        miny = min([miny, min(y)])
        maxy = max([maxy, max(y)])
        minz = min([minz, min(z)])
        maxz = max([maxz, max(z)])
    
        verts = [zip(x, y,z)]
        ax.add_collection3d(Poly3DCollection(verts))
        
    ax.set_xlim3d(minx, maxx)
    ax.set_ylim3d(miny, maxy)
    ax.set_zlim3d(minz, maxz)
    plt.show()